In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid
from sklearn.model_selection import train_test_split
from math import sqrt
import joblib
from datetime import datetime

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

print("All libraries imported successfully.")

All libraries imported successfully.


In [3]:
# Define paths
data_dir = "../data/splits"
train_path = os.path.join(data_dir, "train.csv")
val_path = os.path.join(data_dir, "validation.csv")
test_path = os.path.join(data_dir, "test.csv")

# Load datasets
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

print(f"Train set: {train_df.shape}")
print(f"Validation set: {val_df.shape}")
print(f"Test set: {test_df.shape}")

Train set: (3469, 40)
Validation set: (743, 40)
Test set: (744, 40)


In [4]:
# Target column
target_col = 'price'

# Features: all columns except target
feature_cols = [col for col in train_df.columns if col != target_col]

X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_val = val_df[feature_cols]
y_val = val_df[target_col]

X_test = test_df[feature_cols]
y_test = test_df[target_col]

print("Features and target separated.")
print(f"Number of features: {len(feature_cols)}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")

Features and target separated.
Number of features: 39
Training samples: 3469
Validation samples: 743
Test samples: 744


In [5]:
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, model_name):
    """Calculate metrics on train, validation, and test sets."""
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)

    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    train_mae = mean_absolute_error(y_train, y_train_pred)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    metrics = {
        'Model': model_name,
        'Train RMSE': train_rmse,
        'Validation RMSE': val_rmse,
        'Test RMSE': test_rmse,
        'Train MAE': train_mae,
        'Validation MAE': val_mae,
        'Test MAE': test_mae,
        'Train R2': train_r2,
        'Validation R2': val_r2,
        'Test R2': test_r2
    }

    print(f"\n{'='*50}")
    print(f"{model_name} Performance:")
    print(f"{'='*50}")
    print(f"  Train RMSE: {train_rmse:.4f} | Validation RMSE: {val_rmse:.4f} | Test RMSE: {test_rmse:.4f}")
    print(f"  Train MAE : {train_mae:.4f} | Validation MAE : {val_mae:.4f} | Test MAE : {test_mae:.4f}")
    print(f"  Train R2  : {train_r2:.4f} | Validation R2  : {val_r2:.4f} | Test R2  : {test_r2:.4f}")

    overfit_gap = val_rmse - train_rmse
    test_gap = test_rmse - train_rmse
    print(f"  Overfit Gap (Val-Train): {overfit_gap:.4f}")
    print(f"  Test Gap (Test-Train): {test_gap:.4f}")

    return metrics, y_train_pred, y_val_pred, y_test_pred

In [6]:
print("="*60)
print("OPTUNA TUNING XGBOOST (Validation-based, Strong Regularization)")
print("="*60)

import optuna
from sklearn.metrics import mean_squared_error

def xgb_objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 4),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 0.7),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.7),
        'reg_alpha': trial.suggest_float('reg_alpha', 5, 50, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 50, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 10, 30),
        'gamma': trial.suggest_float('gamma', 0.5, 2.0),
        'n_estimators': 500,
        'random_state': 42,
        'objective': 'reg:squarederror',
        'early_stopping_rounds': 10
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    return rmse

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(xgb_objective, n_trials=30, show_progress_bar=True)

print(f"\nBest XGBoost parameters: {study_xgb.best_params}")
print(f"Best validation RMSE: {study_xgb.best_value:.4f}")

best_xgb_params = study_xgb.best_params.copy()
best_xgb_params['n_estimators'] = 500
best_xgb_params['random_state'] = 42
best_xgb_params['objective'] = 'reg:squarederror'
best_xgb_params['early_stopping_rounds'] = 10

xgb_tuned = xgb.XGBRegressor(**best_xgb_params)
xgb_tuned.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],   # both sets for learning curves
    verbose=False
)

xgb_metrics, xgb_train_pred, xgb_val_pred, xgb_test_pred = evaluate_model(
    xgb_tuned, X_train, y_train, X_val, y_val, X_test, y_test, "XGBoost (Tuned)"
)

models = {'XGBoost (Tuned)': xgb_tuned}

[I 2026-02-16 14:00:30,435] A new study created in memory with name: no-name-cdd56481-3698-4e60-8216-a36430a70315


OPTUNA TUNING XGBOOST (Validation-based, Strong Regularization)


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2026-02-16 14:00:31,899] Trial 0 finished with value: 79.95215155651124 and parameters: {'max_depth': 4, 'learning_rate': 0.0016563509900164785, 'subsample': 0.5889288749460558, 'colsample_bytree': 0.69690631700422, 'reg_alpha': 40.598817491701425, 'reg_lambda': 43.360023285871684, 'min_child_weight': 21, 'gamma': 1.9058695927775855}. Best is trial 0 with value: 79.95215155651124.
[I 2026-02-16 14:00:32,412] Trial 1 finished with value: 59.61843439681585 and parameters: {'max_depth': 3, 'learning_rate': 0.008063477600199888, 'subsample': 0.5953526321097183, 'colsample_bytree': 0.6527958084188059, 'reg_alpha': 36.83119956165535, 'reg_lambda': 7.63237852888659, 'min_child_weight': 23, 'gamma': 1.2653139086606098}. Best is trial 1 with value: 59.61843439681585.
[I 2026-02-16 14:00:33,123] Trial 2 finished with value: 62.341993944060874 and parameters: {'max_depth': 4, 'learning_rate': 0.005377697588692437, 'subsample': 0.5320592697250028, 'colsample_bytree': 0.6571250463544638, 'reg_al

In [7]:
print("\n" + "="*60)
print("OPTUNA TUNING LIGHTGBM (Validation-based, Strong Regularization)")
print("="*60)

def lgb_objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 10, 40),
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 0.7),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.7),
        'reg_alpha': trial.suggest_float('reg_alpha', 1, 20, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 20, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 100),
        'n_estimators': 500,
        'random_state': 42,
        'verbosity': -1
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(10)]
    )
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    return rmse

study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(lgb_objective, n_trials=30, show_progress_bar=True)

print(f"\nBest LightGBM parameters: {study_lgb.best_params}")
print(f"Best validation RMSE: {study_lgb.best_value:.4f}")

best_lgb_params = study_lgb.best_params.copy()
best_lgb_params['n_estimators'] = 500
best_lgb_params['random_state'] = 42
best_lgb_params['verbosity'] = -1

lgb_tuned = lgb.LGBMRegressor(**best_lgb_params)
lgb_tuned.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],   # both sets
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(10)]
)

lgb_metrics, lgb_train_pred, lgb_val_pred, lgb_test_pred = evaluate_model(
    lgb_tuned, X_train, y_train, X_val, y_val, X_test, y_test, "LightGBM (Tuned)"
)

models['LightGBM (Tuned)'] = lgb_tuned

[I 2026-02-16 14:00:48,874] A new study created in memory with name: no-name-b374f8f3-cb51-474a-9c66-0fc98084c1ac



OPTUNA TUNING LIGHTGBM (Validation-based, Strong Regularization)


  0%|          | 0/30 [00:00<?, ?it/s]

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 58.608	valid_0's l2: 3434.9
[I 2026-02-16 14:00:49,369] Trial 0 finished with value: 58.60801629183911 and parameters: {'num_leaves': 27, 'max_depth': 6, 'learning_rate': 0.00529805716495862, 'subsample': 0.690809289323125, 'colsample_bytree': 0.5014558159768551, 'reg_alpha': 9.489371685115119, 'reg_lambda': 3.1624622588523383, 'min_child_samples': 58}. Best is trial 0 with value: 58.60801629183911.
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 69.8841	valid_0's l2: 4883.79
[I 2026-02-16 14:00:49,542] Trial 1 finished with value: 69.88409486873272 and parameters: {'num_leaves': 13, 'max_depth': 3, 'learning_rate': 0.002927112722614055, 'subsample': 0.6413068605320908, 'colsample_bytree': 0.6468216987247428, 'reg_alpha': 3.3867840446247204, 'reg_lambda': 2.7315356658872467, 

In [8]:
print("\n" + "="*60)
print("OPTUNA TUNING CATBOOST (Validation-based, Strong Regularization)")
print("="*60)

def cb_objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.03, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 20, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 1, 3),
        'border_count': trial.suggest_int('border_count', 32, 64),
        'iterations': 500,
        'random_seed': 42,
        'early_stopping_rounds': 10,
        'verbose': False
    }

    model = cb.CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    return rmse

study_cb = optuna.create_study(direction='minimize')
study_cb.optimize(cb_objective, n_trials=30, show_progress_bar=True)

print(f"\nBest CatBoost parameters: {study_cb.best_params}")
print(f"Best validation RMSE: {study_cb.best_value:.4f}")

best_cb_params = study_cb.best_params.copy()
best_cb_params['iterations'] = 500
best_cb_params['random_seed'] = 42
best_cb_params['early_stopping_rounds'] = 10
best_cb_params['verbose'] = False

cb_tuned = cb.CatBoostRegressor(**best_cb_params)
cb_tuned.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)]    # both sets
)

cb_metrics, cb_train_pred, cb_val_pred, cb_test_pred = evaluate_model(
    cb_tuned, X_train, y_train, X_val, y_val, X_test, y_test, "CatBoost (Tuned)"
)

models['CatBoost (Tuned)'] = cb_tuned

[I 2026-02-16 14:00:59,101] A new study created in memory with name: no-name-af8f50c0-054d-4250-a906-3190424088f3



OPTUNA TUNING CATBOOST (Validation-based, Strong Regularization)


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2026-02-16 14:00:59,987] Trial 0 finished with value: 60.8989262131448 and parameters: {'depth': 4, 'learning_rate': 0.009501137844669822, 'l2_leaf_reg': 14.295877763452735, 'bagging_temperature': 2.042360459953155, 'border_count': 53}. Best is trial 0 with value: 60.8989262131448.
[I 2026-02-16 14:01:00,730] Trial 1 finished with value: 74.79931477105445 and parameters: {'depth': 5, 'learning_rate': 0.0021618426276401393, 'l2_leaf_reg': 15.119717396548218, 'bagging_temperature': 1.6242519718110648, 'border_count': 40}. Best is trial 0 with value: 60.8989262131448.
[I 2026-02-16 14:01:01,407] Trial 2 finished with value: 65.63080725948832 and parameters: {'depth': 4, 'learning_rate': 0.004796435993054926, 'l2_leaf_reg': 6.543598196930246, 'bagging_temperature': 1.9552392013134077, 'border_count': 53}. Best is trial 0 with value: 60.8989262131448.
[I 2026-02-16 14:01:02,149] Trial 3 finished with value: 57.81697428380993 and parameters: {'depth': 4, 'learning_rate': 0.016951935504945

In [9]:
# Combine all metrics into a DataFrame
metrics_list = [xgb_metrics, lgb_metrics, cb_metrics]
metrics_df = pd.DataFrame(metrics_list)
metrics_df.set_index('Model', inplace=True)

print("\n" + "="*60)
print("TUNED MODELS COMPARISON")
print("="*60)
print("\nTest Set Performance:")
print(metrics_df[['Test RMSE', 'Test MAE', 'Test R2']].round(4))

# Identify best model based on test RMSE
best_model = metrics_df['Test RMSE'].idxmin()
best_rmse = metrics_df.loc[best_model, 'Test RMSE']
print(f"\n🏆 Best Model by Test RMSE: {best_model} (RMSE: {best_rmse:.4f})")


TUNED MODELS COMPARISON

Test Set Performance:
                  Test RMSE  Test MAE  Test R2
Model                                         
XGBoost (Tuned)     52.2491   35.5702   0.7064
LightGBM (Tuned)    49.6772   33.7795   0.7345
CatBoost (Tuned)    54.3103   34.3698   0.6827

🏆 Best Model by Test RMSE: LightGBM (Tuned) (RMSE: 49.6772)


In [10]:
# Extract evaluation history

# XGBoost
xgb_results = xgb_tuned.evals_result()
xgb_train_rmse = xgb_results['validation_0']['rmse']
xgb_val_rmse = xgb_results['validation_1']['rmse']

# LightGBM
lgb_results = lgb_tuned.evals_result_
print("LightGBM evals_result_ keys:", lgb_results.keys())
lgb_train_key = 'training'
lgb_val_key = [k for k in lgb_results.keys() if k != lgb_train_key][0]
print(f"Using validation key: {lgb_val_key}")
lgb_train_rmse = lgb_results[lgb_train_key]['rmse']
lgb_val_rmse = lgb_results[lgb_val_key]['rmse']

# CatBoost
cb_results = cb_tuned.get_evals_result()
print("CatBoost evals_result keys:", cb_results.keys())
cb_train_rmse = cb_results['learn']['RMSE']
cb_val_rmse = cb_results['validation']['RMSE']

# Plotting function
def plot_learning_curve(ax, train_errors, val_errors, model_name):
    epochs = range(1, len(train_errors)+1)
    ax.plot(epochs, train_errors, label='Train RMSE', color='blue')
    ax.plot(epochs, val_errors, label='Validation RMSE', color='orange')

    best_epoch = np.argmin(val_errors) + 1
    ax.axvline(x=best_epoch, color='green', linestyle='--', alpha=0.7, label=f'Best @ {best_epoch}')

    if best_epoch < len(val_errors):
        ax.axvspan(best_epoch, len(val_errors), alpha=0.2, color='red', label='Overfitting region')

    final_gap = val_errors[-1] - train_errors[-1]
    ax.text(0.05, 0.95, f'Final Gap: {final_gap:.2f}', transform=ax.transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    ax.set_title(f'{model_name} Learning Curve')
    ax.set_xlabel('Boosting Rounds')
    ax.set_ylabel('RMSE')
    ax.legend()
    ax.grid(True, alpha=0.3)

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
plot_learning_curve(axes[0], xgb_train_rmse, xgb_val_rmse, 'XGBoost (Tuned)')
plot_learning_curve(axes[1], lgb_train_rmse, lgb_val_rmse, 'LightGBM (Tuned)')
plot_learning_curve(axes[2], cb_train_rmse, cb_val_rmse, 'CatBoost (Tuned)')
plt.suptitle('Learning Curves with Overfitting Detection', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

LightGBM evals_result_ keys: dict_keys(['training', 'valid_1'])
Using validation key: valid_1
CatBoost evals_result keys: dict_keys(['learn', 'validation_0', 'validation_1'])


KeyError: 'validation'

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# XGBoost
axes[0].plot(xgb_train_rmse, label='Train Loss', color='blue')
axes[0].plot(xgb_val_rmse, label='Validation Loss', color='red')
axes[0].fill_between(range(len(xgb_train_rmse)), xgb_train_rmse, xgb_val_rmse, alpha=0.3, color='gray')
axes[0].set_title('XGBoost Loss Curve')
axes[0].set_xlabel('Iterations')
axes[0].set_ylabel('RMSE')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# LightGBM
axes[1].plot(lgb_train_rmse, label='Train Loss', color='blue')
axes[1].plot(lgb_val_rmse, label='Validation Loss', color='red')
axes[1].fill_between(range(len(lgb_train_rmse)), lgb_train_rmse, lgb_val_rmse, alpha=0.3, color='gray')
axes[1].set_title('LightGBM Loss Curve')
axes[1].set_xlabel('Iterations')
axes[1].set_ylabel('RMSE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# CatBoost
axes[2].plot(cb_train_rmse, label='Train Loss', color='blue')
axes[2].plot(cb_val_rmse, label='Validation Loss', color='red')
axes[2].fill_between(range(len(cb_train_rmse)), cb_train_rmse, cb_val_rmse, alpha=0.3, color='gray')
axes[2].set_title('CatBoost Loss Curve')
axes[2].set_xlabel('Iterations')
axes[2].set_ylabel('RMSE')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle('Training vs Validation Loss Curves', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
residuals = {
    'XGBoost (Tuned)': y_test - xgb_test_pred,
    'LightGBM (Tuned)': y_test - lgb_test_pred,
    'CatBoost (Tuned)': y_test - cb_test_pred
}

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, (name, resid) in enumerate(residuals.items()):
    # Scatter
    ax = axes[0, i]
    ax.scatter(y_test, resid, alpha=0.5, s=20)
    ax.axhline(y=0, color='red', linestyle='--')
    ax.set_title(f'{name} Residuals')
    ax.set_xlabel('Actual Price')
    ax.set_ylabel('Residuals')
    ax.grid(True, alpha=0.3)

    # Histogram
    ax = axes[1, i]
    ax.hist(resid, bins=30, edgecolor='black', alpha=0.7)
    ax.axvline(x=0, color='red', linestyle='--')
    ax.set_title(f'{name} Residual Distribution')
    ax.set_xlabel('Residuals')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

    mean_resid = np.mean(resid)
    std_resid = np.std(resid)
    ax.text(0.05, 0.95, f'Mean: {mean_resid:.2f}\nStd: {std_resid:.2f}',
            transform=ax.transAxes, fontsize=9, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.suptitle('Residual Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
predictions = {
    'XGBoost (Tuned)': xgb_test_pred,
    'LightGBM (Tuned)': lgb_test_pred,
    'CatBoost (Tuned)': cb_test_pred
}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (name, pred) in zip(axes, predictions.items()):
    ax.scatter(y_test, pred, alpha=0.5, s=20)
    min_val = min(y_test.min(), pred.min())
    max_val = max(y_test.max(), pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect')
    r2 = r2_score(y_test, pred)
    ax.set_title(f'{name}\nActual vs Predicted (R² = {r2:.4f})')
    ax.set_xlabel('Actual Price')
    ax.set_ylabel('Predicted Price')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Test Set: Actual vs Predicted', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
from math import pi

# Prepare metrics for radar (normalize)
metrics_for_radar = ['Test RMSE', 'Test MAE', 'Test R2']
xgb_vals = [xgb_metrics['Test RMSE'], xgb_metrics['Test MAE'], xgb_metrics['Test R2']]
lgb_vals = [lgb_metrics['Test RMSE'], lgb_metrics['Test MAE'], lgb_metrics['Test R2']]
cb_vals = [cb_metrics['Test RMSE'], cb_metrics['Test MAE'], cb_metrics['Test R2']]

# Normalize (1 is best for all after transformation)
max_rmse = max(xgb_vals[0], lgb_vals[0], cb_vals[0])
max_mae = max(xgb_vals[1], lgb_vals[1], cb_vals[1])
xgb_norm = [1 - xgb_vals[0]/max_rmse, 1 - xgb_vals[1]/max_mae, xgb_vals[2]]
lgb_norm = [1 - lgb_vals[0]/max_rmse, 1 - lgb_vals[1]/max_mae, lgb_vals[2]]
cb_norm = [1 - cb_vals[0]/max_rmse, 1 - cb_vals[1]/max_mae, cb_vals[2]]

N = len(metrics_for_radar)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

xgb_norm += xgb_norm[:1]
lgb_norm += lgb_norm[:1]
cb_norm += cb_norm[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
ax.plot(angles, xgb_norm, 'o-', linewidth=2, label='XGBoost', color='blue')
ax.fill(angles, xgb_norm, alpha=0.1, color='blue')
ax.plot(angles, lgb_norm, 'o-', linewidth=2, label='LightGBM', color='green')
ax.fill(angles, lgb_norm, alpha=0.1, color='green')
ax.plot(angles, cb_norm, 'o-', linewidth=2, label='CatBoost', color='orange')
ax.fill(angles, cb_norm, alpha=0.1, color='orange')

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics_for_radar, fontsize=12)
ax.set_ylim(0, 1)
ax.set_title('Model Comparison Radar Chart\n(Higher is better)', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
plt.tight_layout()
plt.show()

In [ ]:
# Assess overfitting and accuracy
def assess_overfitting(train_rmse, val_rmse, test_rmse):
    val_gap = val_rmse - train_rmse
    test_gap = test_rmse - train_rmse
    val_gap_pct = (val_gap / train_rmse) * 100
    test_gap_pct = (test_gap / train_rmse) * 100
    avg_gap_pct = (val_gap_pct + test_gap_pct) / 2
    if avg_gap_pct < 5:
        return "LOW", "✅ Generalizes very well"
    elif avg_gap_pct < 15:
        return "MODERATE", "⚠️ Some overfitting, acceptable"
    else:
        return "HIGH", "❌ Significant overfitting"

def accuracy_rating(r2):
    if r2 >= 0.9:
        return "EXCELLENT", "🌟 Very high accuracy"
    elif r2 >= 0.8:
        return "GOOD", "👍 Good predictive power"
    elif r2 >= 0.6:
        return "FAIR", "⚡ Moderate accuracy"
    elif r2 >= 0.4:
        return "POOR", "⚠️ Low accuracy"
    else:
        return "VERY POOR", "❌ Needs improvement"

# Collect details
model_details = []
for model_name, model in models.items():
    if 'XGBoost' in model_name:
        train_rmse = xgb_metrics['Train RMSE']
        val_rmse = xgb_metrics['Validation RMSE']
        test_rmse = xgb_metrics['Test RMSE']
        test_r2 = xgb_metrics['Test R2']
        test_mae = xgb_metrics['Test MAE']
    elif 'LightGBM' in model_name:
        train_rmse = lgb_metrics['Train RMSE']
        val_rmse = lgb_metrics['Validation RMSE']
        test_rmse = lgb_metrics['Test RMSE']
        test_r2 = lgb_metrics['Test R2']
        test_mae = lgb_metrics['Test MAE']
    else:
        train_rmse = cb_metrics['Train RMSE']
        val_rmse = cb_metrics['Validation RMSE']
        test_rmse = cb_metrics['Test RMSE']
        test_r2 = cb_metrics['Test R2']
        test_mae = cb_metrics['Test MAE']

    overfit_level, overfit_desc = assess_overfitting(train_rmse, val_rmse, test_rmse)
    acc_rating, acc_desc = accuracy_rating(test_r2)

    model_details.append({
        'Model': model_name,
        'Train RMSE': train_rmse,
        'Val RMSE': val_rmse,
        'Test RMSE': test_rmse,
        'Test MAE': test_mae,
        'Test R²': test_r2,
        'Overfitting Level': overfit_level,
        'Overfitting Description': overfit_desc,
        'Accuracy Rating': acc_rating,
        'Accuracy Description': acc_desc
    })

summary_df = pd.DataFrame(model_details)

print("="*80)
print("📊 FINAL TUNED MODELS SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))

# Composite score for best model selection
summary_df['RMSE_norm'] = 1 - (summary_df['Test RMSE'] / summary_df['Test RMSE'].max())
summary_df['R2_norm'] = summary_df['Test R²'] / summary_df['Test R²'].max()
overfit_map = {'LOW': 1.0, 'MODERATE': 0.6, 'HIGH': 0.2}
summary_df['Overfit_score'] = summary_df['Overfitting Level'].map(overfit_map)
summary_df['Composite'] = 0.4*summary_df['R2_norm'] + 0.3*summary_df['RMSE_norm'] + 0.3*summary_df['Overfit_score']

best_idx = summary_df['Composite'].idxmax()
best_model = summary_df.loc[best_idx]

print(f"\n🏆 BEST MODEL (by composite score): {best_model['Model']}")
print(f"   Test RMSE: {best_model['Test RMSE']:.4f}, Test R²: {best_model['Test R²']:.4f}")
print(f"   Overfitting: {best_model['Overfitting Level']}, Accuracy: {best_model['Accuracy Rating']}")

# Save tuned models
models_dir = "../models/tuned_models"
os.makedirs(models_dir, exist_ok=True)

for model_name, model in models.items():
    safe_name = model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()
    filename = f"{safe_name}.joblib"
    joblib.dump(model, os.path.join(models_dir, filename))
    print(f"✅ Saved: {filename}")

# Save metadata
metadata = summary_df[['Model', 'Test RMSE', 'Test MAE', 'Test R²', 'Overfitting Level', 'Accuracy Rating']].copy()
metadata['Best_Model'] = best_model['Model']
metadata['Generated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
metadata.to_csv(os.path.join(models_dir, 'model_metadata.csv'), index=False)
print(f"✅ Saved model metadata")

print(f"\n📁 All tuned models saved to: {os.path.abspath(models_dir)}")